In [98]:
from helper import *
from funcs import *

**tf.tsv**

always predict A-coref as True and B-coref as False

In [2]:
f_test = open("gap-test.tsv", "r")
result = open("tf.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    result.write("\t".join([attrs[0]] + ['True', 'False']) + "\n")

f_test.close()
result.close()

In [3]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=tf.tsv

Overall recall: 51.8 precision: 45.9 f1: 48.7
		tp 918	fp 1082
		fn 855	tn 1145
Masculine recall: 51.0 precision: 45.3 f1: 48.0
		tp 453	fp 547
		fn 436	tn 564
Feminine recall: 52.6 precision: 46.5 f1: 49.4
		tp 465	fp 535
		fn 419	tn 581
Bias (F/M): 1.03

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:


**closer.tsv**

Predict closer coreference to pronoun as True

In [4]:
f_test = open("gap-test.tsv", "r")
result = open("closer.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    tf = ['True', 'False']
    if abs(int(attrs[3]) - int(attrs[5])) > abs(int(attrs[3]) - int(attrs[8])):
        tf = ['False', 'True']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [5]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=closer.tsv

Overall recall: 50.1 precision: 44.4 f1: 47.1
		tp 888	fp 1112
		fn 885	tn 1115
Masculine recall: 51.6 precision: 45.9 f1: 48.6
		tp 459	fp 541
		fn 430	tn 570
Feminine recall: 48.5 precision: 42.9 f1: 45.5
		tp 429	fp 571
		fn 455	tn 545
Bias (F/M): 0.94

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:


In [6]:
print_sents(random=True)

Among the mourners were major politicians from both the blue and green camps such as Wang Jin-pyng of the Kuomintang and Ker Chien-Ming of the Democratic Progressive Party, as well as various celebrities of whom the most prominent was popular singer Jay Chou; they suffered harsh criticism for their attendance, including a Taipei Times editorial, which characterised the politicians' presence as ``revolting'' and stated that Chou ``should be ashamed, but we are not sure if he has the depth of character to feel it.'' Chou, who showed up wearing sunglasses and left after only 20 minutes, had become acquainted with Chen through his son Baron Chen, with whom Chou had previously worked in the filming of Kung Fu Dunk.

Due to their father's long-lasting sexual and emotional relationship with his uncle Valdemar, Peter and his sister Eug*nie referred to Valdemar as ``Papa Two''. As customary, Princess George took no part in her son's upbringing, and when he reached adolescence, only the counsels

In [7]:
import pandas as pd

In [8]:
train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data = pd.read_csv('gap-test.tsv', sep='\t')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              2000 non-null   object
 1   Text            2000 non-null   object
 2   Pronoun         2000 non-null   object
 3   Pronoun-offset  2000 non-null   int64 
 4   A               2000 non-null   object
 5   A-offset        2000 non-null   int64 
 6   A-coref         2000 non-null   bool  
 7   B               2000 non-null   object
 8   B-offset        2000 non-null   int64 
 9   B-coref         2000 non-null   bool  
 10  URL             2000 non-null   object
dtypes: bool(2), int64(3), object(6)
memory usage: 144.7+ KB


**subject.tsv**

Predict farther coreference pronoun as True when pronoun is subject pronoun (he, she).
Predict closer coreference pronoun as True when pronoun is not subject pronoun, which is object or possessive pronoun(her, his, him).

In [9]:
pronouns = set(train_data['Pronoun'])
pronouns

{'He', 'Her', 'His', 'She', 'he', 'her', 'him', 'his', 'she'}

In [10]:
result = open("subject.tsv", 'w')

for attrs in test_data.iloc:
    pro, pro_id = attrs[2:4]
    A, A_id = attrs[4:6]
    B, B_id = attrs[7:9]

    if pro.lower() in ['he', 'she']:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['TRUE', 'FALSE']
        else:
            tf = ['FALSE', 'TRUE']
    else:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['FALSE', 'TRUE']
        else:
            tf = ['TRUE', 'FALSE']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [11]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=subject.tsv

Overall recall: 58.3 precision: 51.6 f1: 54.8
		tp 1033	fp 967
		fn 740	tn 1260
Masculine recall: 59.4 precision: 52.8 f1: 55.9
		tp 528	fp 472
		fn 361	tn 639
Feminine recall: 57.1 precision: 50.5 f1: 53.6
		tp 505	fp 495
		fn 379	tn 621
Bias (F/M): 0.96

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:


In [12]:
false_case = 0
for attrs in test_data.iloc:
    if attrs[6] == attrs[9]:
        false_case += 1

false_case/len(test_data)

0.1135

# subject.tsv

draw a relation between two proper nouns with sentence structure

(e.g. Which one is the subject?)

In [13]:
import nltk
sent = "In the next pair, Mikhaylov equalled Grishin's time and they shared Olympic gold."
tagged_sent = nltk.pos_tag(nltk.word_tokenize(sent))

Classify train_data into several categories

1. When words are inside parenthesis
2. When words appear after the pronoun
 * When the pronoun is subject pronoun
 * When the pronoun is object/possessive pronoun
3. When the both coreferences are false
4. When only one word is subject
5. When the both words are subject
6. When none of the words are subject
    

In [14]:
import pandas as pd 

train_data = pd.read_csv('gap-development.tsv', sep='\t')

## 1. When words are inside parenthesis
Assume when word is inside parenthesis, coreference is always false

In [15]:
parenthesis_data = []

def isparenthesis(sent, word, offset):
    if sent[offset-1] == "(" and sent[offset+len(word)] == ")":
        return True
    else:
        return False

for row in train_data.iloc:
    if isparenthesis(row['Text'], row['A'], row['A-offset']) or \
       isparenthesis(row['Text'], row['B'], row['B-offset']):
            parenthesis_data.append(row)

In [16]:
parenthesis_data = pd.DataFrame(parenthesis_data, columns=train_data.columns)

In [17]:
print_sents(datas={'manual':True, 'data':parenthesis_data}, num_lines = 10)

He was named to the NCAA All-Tournament team in 1980, and two first All Star teams in 1981. He began his playing career with the Pittsburgh Penguins who drafted him in the 6th round of the 1979 NHL Entry Draft. Chorney was traded to the Los Angeles Kings for a 6th round choice (Stuart Marston) in the 1985 NHL Entry Draft. After spending a season with the Kings, he was signed by the Washington Capitals but failed to earn a spot on the roster.

Grant played the part in Trevor Nunn's movie adaptation of Twelfth Night, dancing dreadfully in front of Mel Smith's Sir Toby, getting his sword caught in a branch of a tree and being kicked in the groin during his duel with ``Cesario'' (Imogen Stubbs). When Maria (Imelda Staunton) tricks Malvolio (Nigel Hawthorne) into wearing yellow stockings, she reveals to her fellow conspirators that ``'tis a colour (my lady Olivia) abhors''.

When Herman and Fred are arrested and imprisoned for petty theft charges, Ma takes over the group and leads Arthur an

result : 22 out of 31 sentences are predicted correct --> accuracy : 71%

## 2. When words appear after the pronoun
Assume coreference of words coming after the pronoun is false

In [18]:
after_pronoun = []

for row in train_data.iloc:
    if row['Pronoun-offset'] < row['A-offset'] or row['Pronoun-offset'] < row['B-offset']:
        after_pronoun.append(row)

after_pronoun = pd.DataFrame(after_pronoun, columns=train_data.columns)

In [19]:
after_subject_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['he', 'she', 'He', 'She'])]
after_other_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['his', 'her', 'him', 'His', 'Her', 'Him'])]

In [20]:
print_sents(datas={'manual':True, 'data':after_subject_pronoun}, num_lines = 10)

Lynn wrote a letter to the other woman, who promptly showed it to Doolittle, who confronted Lynn, telling her to stay out of his business and that any love he had for her was gone. Lynn wrote that his statement broke her heart, and the experience eventually led her to write ``Fist City'' and the similar ``You Ain't Woman Enough''. Admitting that she was tenaciously jealous and Doolittle sometimes did not spurn the advances of other women, Lynn penned the song as a warning, later reflecting in her 2001 autobiography, ``I've been in a couple of fights in my life.

However, as he skids across the ``rink'', he crashes into a closet and comes across a pair of ice skates. The chase resumes as Tom stumbles across many kitchen obstacles, including an ironing board, a door and some stools, before sliding up a table-come-ramp and falling down into the cellar. Tom emerges from the cellar and just as he is about to catch Jerry, Nibbles defrosts the ice, causing Tom to slip over on the watery floor

In [21]:
def after_pronoun_predict(data):
    A_pred = []
    B_pred = []
    for row in data.iloc:
        A_value = False if row['A-offset'] > row['Pronoun-offset'] else True
        B_value = False if row['B-offset'] > row['Pronoun-offset'] else True

        A_pred.append(A_value)
        B_pred.append(B_value)

    data['A-pred'] = A_pred
    data['B-pred'] = B_pred

In [22]:
after_pronoun_predict(after_subject_pronoun)
after_pronoun_predict(after_other_pronoun)

<ipython-input-21-ceb8dec2fe17>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['A-pred'] = A_pred
<ipython-input-21-ceb8dec2fe17>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['B-pred'] = B_pred


In [23]:
print(accuracy(after_subject_pronoun))
print(accuracy(after_other_pronoun))

0.411214953271028
0.2133676092544987


Result : It doesn't really matter whether a word comes after the pronoun or not

## 3. When the both word coreference is false

Let's find out what they have in common.

In [24]:
false_cases = 0
for row in train_data.iloc:
    if row['A-coref'] == row['B-coref']:
        false_cases += 1

print(false_cases/len(train_data))
print(false_cases)

0.1005
201


There are more than 10 percent of these kind of cases

In [25]:
falses_data = train_data[(train_data['A-coref'] == False) & (train_data['B-coref'] == False)]

In [26]:
print_sents(datas={'manual':True, 'data':falses_data}, num_lines = 10)

When Gao received this news, he became even more respectful of Liang. In the aftermaths of Emperor Zhuangzong's death, Emperor Zhuangzong's adoptive brother Li Siyuan claimed imperial title (as Emperor Mingzong). Meanwhile, Gao Jixing prepared his fleet and wanted to attack Chu. His official Sun Guangxian, whom Liang Zhen had recommended to him to be his scribe, pointed out that Jingnan had barely recovered from the wars, and if it were engaged in a war with Chu, other states might take advantage and attack Jingnan.

In order to get hold of Bobby Z after all, Don Huertero takes a colleague of Tad Gruzsa as hostage and proposes an exchange. Bobby Z knows Don Huertero won't rest until he believes him dead. Being worried sick, he bribes Tad Gruzsa. Now Tad Gruzsa conceives a plan to deceive Don Huertero. He wants to make Don Huertero believe Bobby Z was dead without harming the real Bobby Z. When the exchange is supposed to take place, Tad Gruzsa replaces Bobby Z. In his stead the clueles

## 4. When only one word is subject
Assume that subject word is always true, and other word is always false

In [27]:
subject_data = train_data[train_data['Pronoun'].isin(['he', 'she', 'He', 'She'])]
other_data = train_data[train_data['Pronoun'].isin(['his', 'him', 'her', 'His', 'Him', 'Her'])]
print(len(subject_data))
print(len(other_data))

801
1199


In [97]:
only_subject_data = apply_model(subject_data, [only_subject], word_subject)

462 0.8246753246753247


In [95]:
none_subject_data = apply_model(subject_data, [both_subject, none_different_sents], latter_word)

27 0.7037037037037037


In [96]:
print_sents(datas={'manual':True, 'data':both_subject_data}, num_lines = 10)

Although she publicly supported the policy of VVD minister Rita Verdonk to limit immigration, privately she was not supportive, as she explained in a June 2006 interview for Opzij. This was given after she resigned from Parliament and shortly after she had moved to the United States of America. In parliament, Hirsi Ali had supported the way Verdonk handled the Pasic case, although privately she felt that Pasic should have been allowed to stay.

His various girlfriends have tried to change his outfits: Penny had no success while Stephanie got him to wear dress shirts and sweaters (which he found uncomfortable), and Priya got him to briefly switch to dress shirts, waistcoats and dark slacks; he reverted to his original outfits after she left for India, but started eschewing his hoodies in favor of unbuttoned safari shirts and non-hooded jackets towards the end of the seventh season, a trend that continued into successive seasons.

The song is about leaving the Northern Lights of England.

## 5. When both words are subject
Assume that latter word is True, while the other is False

In [33]:
both_subject_data = train_data.apply(both_subject, axis=1)
both_subject_data = train_data[both_subject_data]
len(both_subject_data)

172

In [34]:
print_sents(datas={'manual':True, 'data':both_subject_data}, num_lines = 10)

The next day, Peggy visits Stacey in hospital and asks her to confess to burning the pub, as a sentence for arson would be less than one for murder, but Stacey refuses. Peggy realises that Stacey needs to look after her daughter so she leaves.

He walks the streets of the ancient cities, try's a food specialty of the region, Giant Spiders, and goes to see old museum full of war memories from the past. Matt travels to America to be taught wrestling for another of his sporting challenges, Konnie spins some plates on sticks with guest, Andrew, and Zoe takes her first pancake challenge!

Clay's group was chosen to maintain electronic equipment during the planned invasion of Japan. After separation from the Army, Clay completed both the Bachelor's (1947) and Masters (1948) degrees in Physics at Kansas State University. At the University of Wisconsin-Madison, his doctoral dissertation under Professor Gibson Winas was entitled, ``Field Strengths and Spectra of High Frequency Gas Discharges.''

In [44]:
both_subject_data.apply(lambda row: latter_word(row)[0], axis=1).iloc[0]


False

In [36]:
accuracy(both_subject_data)

KeyError: 'A-pred'

### * And when both words are not subject

In [39]:
def none_subject(row):
    return word_subject(row) == (False, False)

In [40]:
none_subject_data = subject_data.apply(none_subject, axis=1)
none_subject_data = subject_data[none_subject_data]
len(none_subject_data)

285

In [41]:
print_sents(datas={'manual':True, 'data':none_subject_data}, num_lines = 10)

She is a loving heart trapped beneath an ugly facade, ignored and unappreciated by those who fail to see beyond what is skin-deep. Bakekang is a woman whose only desire is to feel a little affection from the world that has deprived her of everything she has always longed for. Through Kristal, Bakekang will feel the kind of attention she longs for.

Catherine Marshall, the widow of Dr. Peter Marshall when she wrote the book, following up on her best seller ``A Man Called Peter,'' has been quoted as saying the book was about 75% percent historical, although the main characters (the physician) and mountain woman descended from ancient royalty, are fictionalized. While in the book it is implied she ends up with the doctor, the real ``Christy,'' Catherine Marshall's mother, married a minister.

Her personal best for long jump is 6.55 m set in Bangalore on 5 June 2010 during the second Indian Grand Prix. She became only the fourth Indian woman to cross 6.50 m, joining Anju Bobby George (6.83

In [42]:
none_subject_data['A-pred'] = none_subject_data.apply(lambda row: former_word(row)[0], axis=1)
none_subject_data['B-pred'] = none_subject_data.apply(lambda row: former_word(row)[1], axis=1)

<ipython-input-42-d76347394a96>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  none_subject_data['A-pred'] = none_subject_data.apply(lambda row: former_word(row)[0], axis=1)
<ipython-input-42-d76347394a96>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  none_subject_data['B-pred'] = none_subject_data.apply(lambda row: former_word(row)[1], axis=1)


In [43]:
accuracy(none_subject_data)

0.44912280701754387

## 5. When the pronoun is object/possessive pronoun

### * And when all words are in different sentence with the pronoun

Assume subject words are true, other is false